# Pandas tutorial

In [ ]:
import pandas as pd
import numpy as np

## Data structures

There are two types of data structures in pandas: Series and DataFrames.

**Series**: a pandas Series is a one dimensional data structure (“a one dimensional ndarray”) that can store values — and for every value it holds a unique index.

**DataFrame**: a pandas DataFrame is a two (or more) dimensional data structure – basically a table with rows and columns. The columns have names and the rows have indexes.

#### Create example dataframes

In [ ]:
df = pd.DataFrame({'col one':[100, 200], 'col two':[300, 400]})

In [ ]:
type(df)

In [ ]:
df

In [ ]:
pd.DataFrame(np.random.rand(4, 8))

In [ ]:
pd.DataFrame(np.random.rand(4, 8), columns=list('abcdefgh'))

#### Retrieving dataframe columns

In [ ]:
df

In [ ]:
s = df['col one']

In [ ]:
type(s)

In [ ]:
s

In [ ]:
s.values

In [ ]:
type(s.values)

#### Rename the dataframe columns

In [ ]:
df = df.rename({'col one':'col_one', 'col two':'col_two'}, axis='columns')

In [ ]:
df

Or alternative, if you wish to rename all columns:

In [ ]:
df.columns = ['col1', 'col2']

In [ ]:
df

## Opening data files

You might have your data in .csv files, SQL tables, Excel files, or .tsv files. In all cases, the first step will be to read it into a data structure that’s compatible with pandas. Pandas has a set of utilities for loading various data file types.

In [ ]:
zoo = pd.read_csv('data/zoo.csv')

In [ ]:
zoo

#### Exploring the data

In [ ]:
zoo.shape  # 22 rows and 3 columns

In [ ]:
zoo.head()

In [ ]:
zoo.tail()

In [ ]:
zoo.sample(5)

Let's look at another example: 'test.csv'. The entries in this csv are separated by a semicolon (take a look at the file!)

In [ ]:
df = pd.read_csv('data/article_read.csv', delimiter=';')

In [ ]:
df.head()

Note that this time we didn’t have a header in our csv file, so we should set it up manually. We can add them in as we load the data.

In [ ]:
df = pd.read_csv('data/article_read.csv', delimiter=';', 
                 names=['my_datetime', 'event', 'country', 'user_id', 'source', 'topic'])

In [ ]:
df.head()

## Retrieving data

#### Select specific columns

In [ ]:
df[['country', 'user_id']]

In [ ]:
df.user_id

#### Filter by value

In [ ]:
df[df.source == 'SEO']

Select the user_id, the country and the topic columns for the users who are from country_2! 

In [ ]:
df[df.country == 'country_2'][['user_id','topic', 'country']]

## Data aggregation

In [ ]:
zoo = pd.read_csv('data/zoo.csv', delimiter=',')

In [ ]:
zoo.head()

#### Count the number of rows (the number of animals) in the zoo

In [ ]:
zoo.count()

The `.count()` function counts the number of values in each column. In the case of the zoo dataset, there were 3 columns, and each of them had 22 values in it.

In [ ]:
zoo[['animal']].count()

#### Calculate the total water_need of the animals

You can easily sum the values in the water_need column with `.sum()`:

In [ ]:
zoo.water_need.sum()

In [ ]:
zoo.sum()

This method obviously only makese sense for numerical columns!

#### Find out which is the smallest (and greatest) water_need value

In [ ]:
zoo.water_need.min()

In [ ]:
zoo.water_need.max()

#### Find out which is the average water_need value

In [ ]:
zoo.water_need.mean()

In [ ]:
zoo.water_need.median()

## Grouping

You will often want to do segmentations of the data. For instance, it’s nice to know the mean water_need of all animals as we have computed, but we might want to break this number down by animal types. 

In [ ]:
zoo.groupby('animal').mean()

In [ ]:
zoo.groupby('animal').mean()[['water_need']]  # Returns a Dataframe

In [ ]:
zoo.groupby('animal').mean().water_need  # Returns a Series

We can also group by multiple columns.

In [ ]:
zoo.groupby(['animal', 'uniq_id']).mean()

(In this case, each `animal`-`uniq_id` combination is unique, so no actual grouping takes place.)

## Merging

We often store data in multiple tables (or .csv files) instead of just one. This makes it easier to manage data, avoid redundancy, save disk space, faster queries etc.

In cases like these we may need to pull data from different sources, and this requires a merge.

In [ ]:
zoo_eats = pd.read_csv('data/zoo_eats.csv', delimiter=';')

In [ ]:
zoo_eats

In [ ]:
zoo

Notice the animals are not the same between the two Dataframes.

In [ ]:
zoo.merge(zoo_eats)

We have the same result the other way round:

In [ ]:
zoo_eats.merge(zoo)

There are different ways of merging Dataframes, that handle unique/missing entries differently. 

<img src="png/4-pandas-merge-inner-outer-left-right-1024x771.png" width=500 height=500 align=left>

We can see that what we have done above is an INNER join.

#### Outer join

In [ ]:
zoo.merge(zoo_eats, how = 'outer')

#### Left and right joins

In [ ]:
zoo.merge(zoo_eats, how = 'left')

In [ ]:
zoo.merge(zoo_eats, how = 'right')

Notice that pandas automatically recognised that the `animal` column was the key-column to use for joining the tables.

The key-columns can be explicitly given as follows:

In [ ]:
zoo.merge(zoo_eats, how = 'left', left_on = 'animal', right_on = 'animal')

## Sorting

We can sort a Dataframe by the values in one or more columns using `.sort_values()`.

In [ ]:
zoo.sort_values('water_need')

To sort by more than one column, use the `by` keyword:

In [ ]:
zoo.sort_values(by = ['animal', 'water_need'])

#### Reversing the order

In [ ]:
zoo.sort_values(by = ['water_need'], ascending = False)

Note that the indices are jumbled up after sorting. We can reset the indices as follows:

In [ ]:
zoo.sort_values(by = ['water_need'], ascending = False).reset_index()

Notice that we still retained the old indices in the `index` column. We can choose to drop these using the `drop` keyword if we want:

In [ ]:
zoo.sort_values(by = ['water_need'], ascending = False).reset_index(drop = True)

## Handling NaN values

Recall that left, right and outer merges resulted in `NaN` values.

In [ ]:
zoo.merge(zoo_eats, how = 'left')

We often want to replace the `NaN` values with something more useful. Let's say that in this case we want to replace them with the string value `'unknown'`. The pandas method for doing this operation is `.fillna()`.

In [ ]:
zoo.merge(zoo_eats, how = 'left').fillna('unknown')